# Gestión de Usuarios y Control de Acceso en PostgreSQL

Para muchas de las tareas rutinarias involucradas en la interacción con una base de datos, como leer el contenido de una tabla o agregar nuevas entradas, el superusuario de postgres puede no ser apropiado, ya que elude todas las verificaciones de permisos, lo que conlleva un riesgo inherente. Además, como administrador de la base de datos, casi con seguridad no serás el único que necesitará acceder a la base de datos de alguna manera. Por esta razón, necesitarás una forma de agregar nuevos usuarios a la base de datos y otorgarles los privilegios adecuados que sean apropiados para sus casos de uso.  

## Objetivos

Después de completar este laboratorio, podrás:

- Crear roles en una base de datos y otorgarles permisos de selección
- Crear nuevos usuarios en la base de datos y asignarles el rol apropiado
- Revocar y denegar el acceso a la base de datos de un usuario

## Software utilizado en este Laboratorio

En este laboratorio, usarás PostgreSQL. Es un popular sistema de gestión de bases de datos relacional de código abierto (RDBMS) capaz de realizar una gran cantidad de tareas de administración de bases de datos, como almacenar, manipular, recuperar y archivar datos.

Para completar este laboratorio, accederás al servicio de PostgreSQL a través del IBM Skills Network (SN) Cloud IDE, que es un entorno de desarrollo virtual que utilizarás a lo largo de este curso.

## Base de datos utilizada en este Laboratorio

En este laboratorio, utilizarás una base de datos de [https://postgrespro.com/education/demodb](https://postgrespro.com/education/demodb) distribuida bajo la [licencia de PostgreSQL](https://www.postgresql.org/about/licence/). Almacena un mes de datos sobre vuelos de aerolíneas en Rusia y está organizada según el siguiente esquema:

![Esquema de la base de datos de demostración para PostgreSQL](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/PostgreSQL/Lab%20-%20User%20Management%20and%20Access%20Control/images/DB_schema.png)

## Lanzando PostgreSQL en Cloud IDE

Para comenzar con este laboratorio, lanza PostgreSQL usando el Cloud IDE. Puedes hacerlo siguiendo estos pasos:

1. Haz clic en el botón de **extensión de Skills Network** en el lado izquierdo de la ventana.
    
2. Abre el menú desplegable **DATABASES** y haz clic en **PostgreSQL**.
    
3. Haz clic en el botón **Crear**. PostgreSQL puede tardar unos momentos en iniciarse.
    

![Captura de pantalla para lanzar PostgreSQL](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/SIe6YjlLjbFsXxU-km3VBg/k2-postgre-create.png)

  

## Descargando y Creando la Base de Datos

**Primero, necesitaremos descargar la base de datos.**

1. Abre una nueva terminal haciendo clic en el botón **Nueva Terminal** cerca de la parte inferior de la interfaz.

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/RvGiIJmBE-pV6KzlfQ7ETA/k2-postgre-terminal.png)

2. Ejecuta el siguiente comando en la terminal.
    
    ```bash
    wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/example-guided-project/flights_RUSSIA_small.sql
    ```
    

El archivo que descargaste es una copia de seguridad completa de una base de datos de un mes de datos de vuelos en Rusia. Ahora, puedes realizar una restauración completa del conjunto de datos abriendo primero la CLI de PostgreSQL.

3. Cerca de la parte inferior de la ventana, haz clic en el botón **CLI de PostgreSQL** para lanzar la Interfaz de Línea de Comandos.

![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/B6FH9Nojw_-IePJMUSkIMA/k2-6-postgresqlCLI.png)

4. En la CLI de PostgreSQL, escribe el comando `\i <nombre_del_archivo>.` En tu caso, el nombre del archivo será el nombre del archivo que descargaste, `flights_RUSSIA_small.sql`. Esto restaurará los datos en una nueva base de datos llamada `demo`.
    
    ```bash
    \i flights_RUSSIA_small.sql
    ```
    

Las restauraciones pueden tardar unos momentos en completarse.

5. Verifica que la base de datos se haya creado correctamente ingresando el siguiente comando:
    
    ```bash
    \dt
    ```
    

Deberías ver la siguiente salida mostrando todas las tablas que son parte del esquema `bookings` en la base de datos `demo`.

![La línea de comandos muestra todas las tablas que pertenecen al esquema de reservas en la base de datos demo](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/PostgreSQL/Lab%20-%20User%20Management%20and%20Access%20Control/images/SC_3.png)

# Ejercicio 1: Crear Nuevos Roles y Concederles Privilegios Relevantes

En PostgreSQL, los usuarios, grupos y roles son la misma entidad, con la diferencia de que los usuarios pueden iniciar sesión por defecto.

En este ejercicio, crearás dos nuevos roles: `read_only` y `read_write`, y luego les concederás los privilegios relevantes.

Para comenzar, asegúrate de tener abierta la Interfaz de Línea de Comandos de PostgreSQL y conectada a la base de datos `demo`, de la siguiente manera:

![PostgreSQL CLI](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/PostgreSQL/Lab%20-%20User%20Management%20and%20Access%20Control/images/pg_CLI.png)

## Tarea A: Crear un rol `read_only` y concederle privilegios

1. Para crear un nuevo rol llamado `read_only`, ingresa el siguiente comando en la CLI:
    
    ```bash
    CREATE ROLE read_only;
    ```
    
2. Primero, este rol necesita el privilegio para conectarse a la base de datos `demo`. Para conceder este privilegio, ingresa el siguiente comando en la CLI:
    
    ```bash
    GRANT CONNECT ON DATABASE demo TO read_only;
    ```
    
3. A continuación, el rol necesita poder usar el esquema en uso en esta base de datos. En nuestro ejemplo, este es el esquema `bookings`. Concede el privilegio para que el rol `read_only` use el esquema ingresando lo siguiente:
    
    ```bash
    GRANT USAGE ON SCHEMA bookings TO read_only;
    ```
    
4. Para acceder a la información en las tablas de una base de datos, se utiliza el comando `SELECT`. Para el rol `read_only`, queremos que pueda acceder al contenido de la base de datos, pero no editarlo o alterarlo. Así que para este rol, solo se necesita el privilegio `SELECT`. Para conceder este privilegio, ingresa el siguiente comando:
    
    ```bash
    1GRANT SELECT ON ALL TABLES IN SCHEMA bookings TO read_only;Copied!Wrap Toggled!
    ```
    
    Esto permite que el rol `read_only` ejecute el comando `SELECT` en **todas** las tablas en el esquema bookings.
    

## Tarea B: Crear un rol `read_write` y concederle privilegios

1. De manera similar, crea un nuevo rol llamado `read_write` con el siguiente comando en la CLI de PostgreSQL:
    
    ```bash
    CREATE ROLE read_write;
    ```
    
2. Al igual que en la Tarea A, este rol debe recibir primero los privilegios para conectarse a la base de datos `demo`. Concede este privilegio ingresando el siguiente comando:
    
    ```bash
    GRANT CONNECT ON DATABASE demo TO read_write;
    ```
    
3. Concede al rol los privilegios para usar el esquema `bookings` que se utiliza en la base de datos `demo` con lo siguiente:
    
    ```bash
    GRANT USAGE ON SCHEMA bookings TO read_write;
    ```
    
4. Hasta ahora, los comandos para el rol `read_write` han sido esencialmente los mismos que para el rol `read_only`. Sin embargo, el rol `read_write` debe tener los privilegios no solo para acceder al contenido de la base de datos, sino también para crear, eliminar y modificar entradas. Los comandos correspondientes para estas acciones son `SELECT`, `INSERT`, `DELETE` y `UPDATE`, respectivamente. Concede a este rol estos privilegios ingresando el siguiente comando en la CLI:
    
    ```bash
    GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA bookings TO read_write;
    ```

# Ejercicio 2: Agregar un Nuevo Usuario y Asignarle un Rol Relevante

En este ejercicio, crearás un nuevo usuario para la base de datos y le asignarás uno de los roles que creaste en el Ejercicio 1. Este método agiliza el proceso de agregar nuevos usuarios a la base de datos, ya que no tienes que pasar por el proceso de otorgar privilegios personalizados a cada uno. En su lugar, puedes asignarles un rol y el usuario hereda los privilegios de ese rol.

Supongamos que deseas agregar un nuevo usuario, `user_a`, para ser utilizado por un servicio de información y ayuda en un aeropuerto. En este caso, asume que no es necesario que este usuario modifique el contenido de la base de datos. Como habrás adivinado, el rol apropiado para asignar es el rol `read_only`.

1. Para crear un nuevo usuario llamado `user_a`, ingresa el siguiente comando en la CLI de PostgreSQL:
    
    ```bash
    CREATE USER user_a WITH PASSWORD 'user_a_password';
    ```
    
    En la práctica, ingresarías una contraseña segura en lugar de ‘user\_a\_password’, que se utilizará para acceder a la base de datos a través de este usuario.
    
2. A continuación, asigna a `user_a` el rol `read_only` ejecutando el siguiente comando en la CLI:
    
    ```bash
    GRANT read_only TO user_a;
    ```
    
3. Puedes listar todos los roles y usuarios escribiendo el siguiente comando:
    
    ```bash
    \du
    ```
    
    Verás la siguiente salida:  
    ![Salida de la CLI de PostgreSQL](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/PostgreSQL/Lab%20-%20User%20Management%20and%20Access%20Control/images/UAM_1.png)  
    Observa que `user_a` fue creado con éxito y que es miembro de `read_only`.

# Ejercicio 3: Revocar y Denegar Acceso

En este ejercicio, aprenderás cómo revocar el privilegio de un usuario para acceder a tablas específicas en una base de datos.

Supongamos que no es necesario que la información y el servicio de atención al cliente en el aeropuerto accedan a la información almacenada en la tabla `aircrafts_data`. En este ejercicio, revocarás el privilegio `SELECT` en la tabla `aircrafts_data` en la base de datos `demo` de `user_a`.

  

1. Puedes usar el comando `REVOKE` en la Interfaz de Línea de Comandos para eliminar privilegios específicos de un rol o usuario en PostgreSQL. Ingresa el siguiente comando en la CLI de PostgreSQL para eliminar los privilegios de acceso a la tabla `aircrafts_data` de `user_a`:
    
    ```bash
    REVOKE SELECT ON aircrafts_data FROM user_a;
    ```
    
2. Ahora supongamos que `user_a` es transferido a otro departamento dentro del aeropuerto y ya no necesita poder acceder a la base de datos `demo`. Puedes eliminar todos sus privilegios `SELECT` simplemente revocando el rol `read_only` que le asignaste anteriormente. Puedes hacerlo ingresando el siguiente comando en la CLI:
    
    ```bash
    REVOKE read_only FROM user_a;
    ```
    
3. Ahora puedes verificar todos los usuarios y sus roles nuevamente para ver que el rol `read_only` fue revocado con éxito de `user_a` ingresando el siguiente comando nuevamente:
    
    ```bash
    \du
    ```
    
    Verás la siguiente salida:
    
    ![La CLI de PostgreSQL muestra los resultados de revocar los privilegios de solo lectura](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0231EN-SkillsNetwork/labs/PostgreSQL/Lab%20-%20User%20Management%20and%20Access%20Control/images/UAM_2.png)
    
    Observa que `user_a` sigue presente, pero ya no es miembro del rol `read_only`.

# Ejercicio Práctico

¡Ahora es el momento de implementar parte de lo que aprendiste! En este ejercicio práctico, utilizarás lo que aprendiste en los ejercicios anteriores para crear un nuevo usuario y asignarle un rol relevante.

**Escenario**: _Supongamos que hay un nuevo empleado en la aerolínea para la que eres el administrador de la base de datos. Ellos interactúan directamente con los clientes para crear nuevas reservas de vuelos. Como tal, necesitarán no solo acceder a la información en la base de datos, sino también crear nuevas reservas._

Para completar este ejercicio, crea un nuevo usuario llamado `user_b` y otórgale los privilegios para leer y escribir en la base de datos `demo`.

<details><summary>Pista (Haz clic aquí)</summary>


- Para recordar cómo crear un usuario, siéntete libre de revisar el Ejercicio 2.
- Recuerda que en este laboratorio, creaste un rol `read_write`. Este podría ser un rol apropiado para asignar a `user_b`.
</details>
<br>
<details><summary>Solución (Haz clic aquí)</summary>

1. Primero, puedes crear un nuevo usuario utilizando el siguiente comando:
    
    ```bash
    CREATE USER user_b WITH PASSWORD 'user_b_password';
    ```
    
2. A continuación, puedes otorgar al usuario que acabas de crear el rol `read_write` ingresando el siguiente comando en la CLI:
    
    ```bash
    GRANT read_write TO user_b;
    ```
</details>

# Conclusión

Felicitaciones por completar este laboratorio sobre gestión de usuarios y control de acceso en PostgreSQL. Ahora tienes algunos conocimientos fundamentales sobre cómo crear nuevos roles para tu base de datos, agregar nuevos usuarios y asignar a esos usuarios roles relevantes. Además, también tienes la capacidad de revocar privilegios a los usuarios en la base de datos.